<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scrap" data-toc-modified-id="Scrap-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scrap</a></span><ul class="toc-item"><li><span><a href="#Emoji-data" data-toc-modified-id="Emoji-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Emoji data</a></span></li><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Word2vec</a></span></li></ul></li></ul></div>

In [ ]:

# frequently used builtins
import os
from collections import Counter, defaultdict

# frequently used data pkgs
import numpy as np
import pandas as pd
			
# careful import of matplotlib
try:
	import matplotlib.pyplot as plt
except ModuleNotFoundError as e:  # to catch ModuleNotFoundError: No module named '_tkinter'
	import matplotlib
	matplotlib.use('agg')
	import matplotlib.pyplot as plt

# some magic stuff
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format='retina'




In [1]:
from functools import lru_cache

from idiom import WordVec, WordVecSearch, WordVecStore

@lru_cache(maxsize=1)
def get_word_vec(*args):
    return WordVec(*args)

# Scrap

Below is just some pieces of code written while developing the package.

I might not work, or be the in the right order, or anything. It's scrap. 

I'm sharing it only for those readers (like myself) that like to have a trace of the process that lead to the result.

## Emoji data

In [2]:
import re
from collections import defaultdict

from moji.util import data_dir
from idiom import WordVec, WordVecSearch

In [3]:
list(data_dir.iterdir())

[PosixPath('/Users/Thor.Whalen/Dropbox/dev/p3/proj/t/moji/moji/data/emoji_joined.tsv'),
 PosixPath('/Users/Thor.Whalen/Dropbox/dev/p3/proj/t/moji/moji/data/emoji2vec.tsv')]

In [12]:
f = (data_dir / 'emoji_joined.tsv')

tokenizer = re.compile('\w+').findall

words_of_emo = defaultdict(list)
terms_of_emo = defaultdict(list)
emo_of_description = dict(np.genfromtxt(str(f), delimiter='\t', dtype=str, skip_header=1))
for description, emo in emo_of_description.items():
    words_of_emo[emo].extend(tokenizer(description))
    terms_of_emo[emo].append(tokenizer(description))

In [5]:
wv = get_word_vec()
vec_of_emo = {emo: wv.mean_vec(words) for emo, words in words_of_emo.items()}

/Users/Thor.Whalen/Dropbox/dev/p3/proj/t/idiom/idiom/__init__.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([self.vec_of_word.get(w, None) for w in words])


In [15]:
terms

[['full', 'moon', 'with', 'face'],
 ['smiling', 'moon'],
 ['moonface'],
 ['monsters'],
 ['smiley', 'moon']]

In [19]:
wv = get_word_vec()
vecs_of_emo = defaultdict(list)
for emo, terms in terms_of_emo.items():
    for term in terms:
        vec = wv.mean_vec(term)
        if vec is not None:
            vecs_of_emo[emo].append(vec)

/Users/Thor.Whalen/Dropbox/dev/p3/proj/t/idiom/idiom/__init__.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([self.vec_of_word.get(w, None) for w in words])


In [36]:
emo_vec_search = WordVecSearch(word_vec=WordVec()).fit(vec_of_emo)

In [37]:
emo_vec_search('boy')

array([['🍊', '🐜', '🔣', '👨🏻', '🔄', '🇿🇦', '🤒', '🔱', '👸', '🎿']], dtype='<U8')

In [41]:
emo_vec_search('eat')

array([['🐜', '🎿', '👨🏻', '🔄', '🍊', '👼🏻', '👮🏼', '🤒', '📲', '🔱']], dtype='<U8')

In [42]:
emo_vec_search('eat pizza')

array([['🐜', '🎿', '👨🏻', '🔄', '🍊', '👼🏻', '👮🏼', '🤒', '📲', '🔱']], dtype='<U8')

In [43]:
emo_vec_search('smile')

array([['🐜', '🎿', '👨🏻', '🔄', '🍊', '👼🏻', '👮🏼', '🤒', '📲', '🔱']], dtype='<U8')

In [25]:
np.array([1,2,3]).reshape(1, -1)
np.array([[1,2,3], [4,5,6]])
np.ndim(np.array([1,2,3]).reshape(1, -1))

2

In [29]:
len(emo_vec_search.word_vec.vec_of_word)

1607

In [6]:
# emo_wordvecs = WordVec(vec_of_emo)

In [7]:
emo_vec_search = WordVecSearch(word_vec=WordVec(vec_of_emo)).fit()

In [11]:
vec_of_emo['🐜']

array([ 0.0056    , -0.00203333,  0.0061    ,  0.01976667, -0.02926667,
       -0.0146    ,  0.08246667, -0.0734    ,  0.00716667,  0.0198    ,
        0.02363333, -0.003     , -0.00436667,  0.0242    ,  0.02476667,
       -0.01446667,  0.0578    , -0.00083333,  0.03806667,  0.03516667,
       -0.00706667,  0.05533333, -0.0132    ,  0.0285    ,  0.00903333,
        0.01653333,  0.0129    ,  0.0142    ,  0.03093333,  0.02386667,
       -0.01053333,  0.00773333,  0.0558    , -0.00386667,  0.01106667,
       -0.02256667,  0.02846667, -0.00703333, -0.0074    , -0.01543333,
        0.00016667, -0.0671    , -0.00286667, -0.075     ,  0.0045    ,
        0.0108    , -0.02906667, -0.05123333, -0.00386667,  0.01713333,
        0.00963333,  0.02963333,  0.0315    , -0.05596667,  0.00076667,
       -0.02473333,  0.0146    ,  0.03053333, -0.0335    , -0.02543333,
       -0.01323333, -0.04266667,  0.0918    , -0.02243333, -0.0281    ,
       -0.00376667,  0.0163    ,  0.0067    ,  0.00753333,  0.04

In [10]:
emo_vec_search('poo')

array([['🐜', '🎿', '👨🏻', '🔄', '🍊', '👼🏻', '👮🏼', '🤒', '📲', '🔱']], dtype='<U8')

In [23]:
words_of_emo['🤒']

['face', 'with', 'thermometer', 'ill']

In [12]:
vec_of_emo['🌝']

array([-5.13333333e-03, -1.95555556e-03,  1.78888889e-02, -1.16666667e-03,
       -5.97777778e-03, -2.85111111e-02,  1.90333333e-02, -5.79111111e-02,
        1.10222222e-02,  6.75555556e-03,  1.12222222e-02, -6.01111111e-03,
        1.06666667e-03,  1.75222222e-02, -1.67888889e-02, -2.51333333e-02,
        1.87777778e-02, -1.63000000e-02,  2.72555556e-02, -1.46666667e-03,
       -4.33333333e-03,  3.31111111e-03, -2.35555556e-02,  2.98333333e-02,
        2.92222222e-03, -1.05555556e-03,  3.00000000e-03,  4.13333333e-03,
        3.88888889e-03, -7.55555556e-04, -2.31777778e-02,  9.02222222e-03,
        1.39333333e-02, -7.11111111e-04, -1.00000000e-02, -7.81111111e-03,
       -5.16666667e-03, -3.14666667e-02,  8.53333333e-03, -3.44444444e-03,
       -8.78888889e-03, -4.02666667e-02, -1.55555556e-04, -1.30111111e-02,
        1.77666667e-02,  2.06111111e-02,  2.23333333e-03, -7.95555556e-03,
       -1.16666667e-03,  1.18000000e-02,  1.27666667e-02,  3.40000000e-03,
       -8.16666667e-03, -

In [46]:
from csv import DictReader, Dialect, reader

emoji_data = (data_dir / 'emoji_joined.tsv')

with open(f) as csvfile:
    reader = reader(csvfile, dialect='excel-tab')
    t = dict(reader)

In [47]:
t['ballot box with check']

'☑️'

In [49]:
t['baby chicken']

'🐣'

In [58]:
pd.Series(Counter(map(lambda x: len(x.split(' ')), t))).sort_index()

1     2166
2     1114
3      576
4      232
5      327
6       48
7       38
8        5
9        5
10       4
11       4
12       1
dtype: int64

In [61]:
# wv.mean_vec('ballot box with check'.split(' '))

In [63]:
f = (data_dir / 'emoji_joined.tsv')
dict(np.genfromtxt(str(f), delimiter='\t', dtype=str, skip_header=1))
# pd.read_csv(str(f))

{'full moon with face': '🌝',
 'cheese': '🧀',
 'moon': '🌙',
 'smiling moon': '🌝',
 'phase': '🌙',
 'full': '🌕',
 'sky': '🌙',
 'moonface': '🌝',
 'night': '🌙',
 'werewolves': '🌕',
 'monsters': '🌝',
 'anthropomorphic': '🌚',
 'face': '🌚',
 'smiley moon': '🌝',
 'spooky': '🌕',
 'flag for belarus': '🇧🇾',
 'regional indicator symbol letters by': '🇧🇾',
 'belarusian flag': '🇧🇾',
 'vain': '🐩',
 'miniature poodle': '🐩',
 'dog': '🐕',
 'sophisticated': '🐩',
 'showy': '🐩',
 'clip': '🐩',
 'toy poodle': '🐩',
 'standard poodle': '🐩',
 'poodle': '🐩',
 'black scissors': '✂️',
 'cutting': '✂️',
 'double vertical bar': '⏸',
 'light brown bicyclist': '🚴🏼',
 'greek': '♒',
 'stars': '♒',
 'scorpius': '♏',
 'sign': '👍',
 'astrology': '♒',
 'scorpio': '♏',
 'scorpion': '♏',
 'constellation': '♒',
 'zodiac': '♒',
 'cruise': '🚢',
 'cruise ship': '🚢',
 'boat': '🚤',
 'ferry': '⛴',
 'ship': '⚓️',
 'husk': '🌽',
 'kernel': '🌽',
 'popcorn': '🍿',
 'stalk': '🌽',
 'cob': '🌽',
 'iowa': '🌽',
 'ear': '👂',
 'food': '🌰',
 'yellow

## Word2vec

In [1]:
from idiom import WordVec, WordVecSearch, WordVecStore

wv = WordVec()

In [2]:
word_pairs = [
    ('smile', 'happy'),
    ('happy', 'smile'),
    ('happy', 'happiness'),
    ('happy', 'sad'),
    ('happy', 'table'),
    ('happy', 'persom'),
]
for word_pair in word_pairs:
    print("{}, {:<10}: {:0.04f}".format(*word_pair, wv.dist(*word_pair)))

smile, happy     : 0.4576
happy, smile     : 0.4576
happy, happiness : 0.4244
happy, sad       : 0.3100
happy, table     : 0.6791
happy, persom    : 0.4754
